In [11]:
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, LSTM, Dense
from sklearn.model_selection import train_test_split
from sklearn.utils import shuffle
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords
import nltk

nltk.download('wordnet')
nltk.download('stopwords')

# Load the dataset
dataset = pd.read_csv('C:/Users/SOFTCOM/Desktop/nazarene/dataset_swahiliEnglish.csv',encoding='iso-8859-1')  # Replace 'path_to_dataset.csv' with the actual path to your dataset

# Shuffle the dataset
dataset = shuffle(dataset)

# Initialize the lemmatizer and stopwords
lemmatizer = WordNetLemmatizer()
stop_words = set(stopwords.words('english'))

# Lemmatization and lowercase conversion for Swahili column
dataset['swahili'] = dataset['swahili'].apply(lambda x: ' '.join([lemmatizer.lemmatize(word.lower()) for word in str(x).split()]))

# Lemmatization and lowercase conversion for English column
dataset['english'] = dataset['english'].apply(lambda x: ' '.join([lemmatizer.lemmatize(word.lower()) for word in str(x).split()]))

# Split the dataset into training and validation sets
train_data, val_data = train_test_split(dataset, test_size=0.2)

# Get the Swahili and English sentences from the dataset
input_texts = train_data['swahili'].values.tolist()
target_texts = train_data['english'].values.tolist()
val_input_texts = val_data['swahili'].values.tolist()
val_target_texts = val_data['english'].values.tolist()

# Create a set of all unique characters in the input and target texts
input_characters = sorted(list(set(''.join(input_texts))))
target_characters = sorted(list(set(''.join(target_texts))))

# Add the start token '\t' to the target characters set if it is not already present
if '\t' not in target_characters:
    target_characters.append('\t')

# Get the number of unique input and target characters
num_encoder_tokens = len(input_characters) + 1  # Add 1 for the start token '\t'
num_decoder_tokens = len(target_characters) + 1  # Add 1 for the start token '\t'

# Create character-to-index and index-to-character dictionaries for input and target characters
input_char_index = dict([(char, i) for i, char in enumerate(input_characters)])
target_char_index = dict([(char, i) for i, char in enumerate(target_characters)])
reverse_input_char_index = dict((i, char) for char, i in input_char_index.items())
reverse_target_char_index = dict((i, char) for char, i in target_char_index.items())

# Get the maximum sequence lengths for input and target texts
max_encoder_seq_length = max([len(text) for text in input_texts])
max_decoder_seq_length = max([len(text) for text in target_texts])

# Prepare the training data
encoder_input_data = np.zeros((len(input_texts), max_encoder_seq_length, num_encoder_tokens), dtype='float32')
decoder_input_data = np.zeros((len(input_texts), max_decoder_seq_length, num_decoder_tokens), dtype='float32')
decoder_target_data = np.zeros((len(input_texts), max_decoder_seq_length, num_decoder_tokens), dtype='float32')

for i, (input_text, target_text) in enumerate(zip(input_texts, target_texts)):
    for t, char in enumerate(input_text):
        encoder_input_data[i, t, input_char_index[char]] = 1.0
    for t, char in enumerate(target_text):
        decoder_input_data[i, t, target_char_index[char]] = 1.0
        if t > 0:
            decoder_target_data[i, t - 1, target_char_index[char]] = 1.0

# Define the model architecture
encoder_inputs = Input(shape=(None, num_encoder_tokens))
encoder = LSTM(256, return_state=True)
encoder_outputs, state_h, state_c = encoder(encoder_inputs)
encoder_states = [state_h, state_c]

decoder_inputs = Input(shape=(None, num_decoder_tokens))
decoder = LSTM(256, return_sequences=True, return_state=True)
decoder_outputs, _, _ = decoder(decoder_inputs, initial_state=encoder_states)
decoder_dense = Dense(num_decoder_tokens, activation='softmax')
decoder_outputs = decoder_dense(decoder_outputs)

model = Model([encoder_inputs, decoder_inputs], decoder_outputs)

# Compile and train the model
model.compile(optimizer='rmsprop', loss='categorical_crossentropy', metrics=['accuracy'])
model.fit([encoder_input_data, decoder_input_data], decoder_target_data, batch_size=64, epochs=50, validation_split=0.2)

# Define the encoder model to get the encoder states from the trained model
encoder_model = Model(encoder_inputs, encoder_states)

# Define the decoder model
decoder_state_input_h = Input(shape=(256,))
decoder_state_input_c = Input(shape=(256,))
decoder_states_inputs = [decoder_state_input_h, decoder_state_input_c]
decoder_outputs, state_h, state_c = decoder(decoder_inputs, initial_state=decoder_states_inputs)
decoder_states = [state_h, state_c]
decoder_outputs = decoder_dense(decoder_outputs)
decoder_model = Model([decoder_inputs] + decoder_states_inputs, [decoder_outputs] + decoder_states)

# Function to convert input text to input array
def string_to_input_array(input_text):
    input_array = np.zeros((1, max_encoder_seq_length, num_encoder_tokens), dtype='float32')
    for t, char in enumerate(input_text):
        if char in input_char_index:
            input_array[0, t, input_char_index[char]] = 1.0
    return input_array

# Function to translate Swahili to English
def translate_swahili_to_english(input_text):
    # Lemmatization and lowercase conversion
    input_text = ' '.join([lemmatizer.lemmatize(word.lower()) for word in input_text.split()])

    # Stopword removal
    input_text = ' '.join([word for word in input_text.split() if word not in stop_words])

    # Encode the input sentence to get the initial states for the decoder
    states_value = encoder_model.predict(string_to_input_array(input_text))

    # Generate empty target sequence of length 1
    target_seq = np.zeros((1, 1, num_decoder_tokens), dtype='float32')
    target_seq[0, 0, target_char_index['\t']] = 1.0  # Set the start token as the first character

    # Decode the input sequence to get the translated sentence
    translated_sentence = ''
    stop_condition = False
    while not stop_condition:
        output_tokens, h, c = decoder_model.predict([target_seq] + states_value)

        # Sample a token from the output distribution
        sampled_token_index = np.argmax(output_tokens[0, -1, :])
        sampled_char = reverse_target_char_index[sampled_token_index]
        translated_sentence += sampled_char

        # Exit condition: either hit max length or find stop character
        if sampled_char == '\n' or len(translated_sentence) > max_decoder_seq_length:
            stop_condition = True

        # Update the target sequence for the next iteration
        target_seq = np.zeros((1, 1, num_decoder_tokens), dtype='float32')
        target_seq[0, 0, sampled_token_index] = 1.0

        # Update states
        states_value = [h, c]

    return translated_sentence

# Test the translation function
input_text = 'Jambo'
translation = translate_swahili_to_english(input_text)
print(f"Input: {input_text}")
print(f"Translation: {translation}")


[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\SOFTCOM\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\SOFTCOM\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


Epoch 1/50
26/26 [==============================] - 14s 339ms/step - loss: 0.5415 - accuracy: 0.0240 - val_loss: 0.5151 - val_accuracy: 0.0245
Epoch 2/50
26/26 [==============================] - 7s 267ms/step - loss: 0.5130 - accuracy: 0.0258 - val_loss: 0.5062 - val_accuracy: 0.0245
Epoch 3/50
26/26 [==============================] - 7s 280ms/step - loss: 0.5111 - accuracy: 0.0256 - val_loss: 0.5105 - val_accuracy: 0.0245
Epoch 4/50
26/26 [==============================] - 6s 245ms/step - loss: 0.5110 - accuracy: 0.0258 - val_loss: 0.5131 - val_accuracy: 0.0204
Epoch 5/50
26/26 [==============================] - 7s 256ms/step - loss: 0.5109 - accuracy: 0.0256 - val_loss: 0.5116 - val_accuracy: 0.0245
Epoch 6/50
26/26 [==============================] - 7s 258ms/step - loss: 0.5108 - accuracy: 0.0258 - val_loss: 0.5140 - val_accuracy: 0.0245
Epoch 7/50
26/26 [==============================] - 7s 276ms/step - loss: 0.5111 - accuracy: 0.0258 - val_loss: 0.5118 - val_accuracy: 0.0245
Epoch

1/1 [==============================] - 0s 31ms/step
Input: Jambo
Translation: aaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaa
